In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('data/ex3data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [2]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [76]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [60]:
def computeCost(theta,X,y,learningRate):
    theta=np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)
    lamb=learningRate
    first=np.multiply(-y,np.log(sigmoid((X*theta.T))))
    second=np.multiply((1-y),np.log(1-sigmoid(X*theta.T)))
    reg=(lamb/(2*len(X)))*np.sum(np.power(theta,2))
    return np.sum(first-second)/(len(y))+reg

In [5]:
def gradientReg(theta,X,y,learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    lamb=learningRate
    
    parameters=int(theta.ravel().shape[1])
    grad=np.zeros(parameters)
    
    error=sigmoid(X * theta.T)-y
    
    gradNR=((np.matmul(error.T,X))/len(X))
    gradR=gradNR+((lamb/len(X))*theta)
    gradRF=gradR
    gradRF[0]=gradNR[0]
    A = np.squeeze(np.asarray(gradRF))
    return A

In [78]:
from scipy.optimize import minimize

def one_vs_all(X,y,num_labels,learning_rate):
    rows=X.shape[0]
    parameters=X.shape[1]
        
    all_theta=np.zeros((num_labels,parameters+1))
    X=np.insert(X,0,values=np.ones(rows),axis=1)
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        a = np.array([1 if label == i else 0 for label in y])
        a = np.reshape(a, (rows, 1))
        fmin = minimize(fun=computeCost, x0=theta, args=(X, a, learning_rate), method='TNC', jac=gradientReg)
        all_theta[i-1,:] = fmin.x
    return all_theta

In [62]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [70]:
np.unique(data['y'])
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[ -1.44077205e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.47745308e-03,   2.89558413e-07,   0.00000000e+00],
       [ -3.23533732e+00,   0.00000000e+00,   0.00000000e+00, ...,
          3.65672472e-03,  -4.22251147e-04,   0.00000000e+00],
       [ -4.84667571e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -3.28110819e-05,   3.35304811e-07,   0.00000000e+00],
       ..., 
       [ -7.92753720e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.87054947e-05,   6.71064717e-06,   0.00000000e+00],
       [ -4.90819052e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -5.01343220e-04,   3.25097768e-05,   0.00000000e+00],
       [ -3.77627118e+00,   0.00000000e+00,   0.00000000e+00, ...,
          5.43848405e-04,   1.28339518e-05,   0.00000000e+00]])

In [71]:
all_theta.shape

(10, 401)

In [72]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax
type(all_theta)

numpy.ndarray

In [77]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.24%
